In [1]:
import glob
import os
import shutil

from astropy.io import fits
from astropy import stats
from astropy.time import Time
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Determine where the visualization will be rendered
output_notebook()

Loading BokehJS ...

In [2]:
postflash_data = pd.read_pickle('/grp/hst/wfc3u/postflash_2021/flc_all_stats_postflash.pkl')
postflash_data

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,median,sig_median,std,sig_std,norm_by_dur,datetime
rootname,,,,,,,,,,,,,,,,,,,
ic6de2u9q,ic6de2u9q_flc.fits,13078,/grp/hst/wfc3u/postflash_2021/mastDownload/HST...,2013-03-09,13:59:17,B,12,4.7,LOW,True,0.5,7.349365,7.079852,6.930480,6.93048,17.969223,2.949302,1.506351,2013-03-09 13:59:17
icha24imq,icha24imq_flc.fits,13560,/grp/hst/wfc3u/postflash_2021/mastDownload/HST...,2014-06-29,04:16:36,A,3,1.2,LOW,True,0.5,2.202514,2.000973,1.947791,1.94779,15.103395,2.345361,1.667477,2014-06-29 04:16:36
idp530aiq,idp530aiq_flc.fits,14984,/grp/hst/wfc3u/postflash_2021/mastDownload/HST...,2018-06-22,04:55:12,B,15,0.2,MED,True,0.5,9.039907,8.727468,8.900126,8.90013,23.013975,3.231754,43.637340,2018-06-22 04:55:12
ieg845feq,ieg845feq_flc.fits,16398,/grp/hst/wfc3u/postflash_2021/mastDownload/HST...,2020-12-14,20:55:49,B,7310,100.0,MED,False,0.5,4685.844727,4684.119095,4759.512695,4759.51,435.797791,418.421755,46.841191,2020-12-14 20:55:49
icrz19f7q,icrz19f7q_flc.fits,14006,/grp/hst/wfc3u/postflash_2021/mastDownload/HST...,2015-05-25,22:31:24,B,12,4.7,LOW,True,0.5,7.384943,7.056043,7.002403,7.0024,27.685162,3.009128,1.501286,2015-05-25 22:31:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
icrz24ubq,icrz24ubq_flc.fits,14006,/grp/hst/wfc3u/postflash_2021/mastDownload/HST...,2015-06-23,19:35:12,B,5,2.0,LOW,True,0.5,3.206771,2.920654,3.029151,3.02915,20.221544,2.402493,1.460327,2015-06-23 19:35:12
ieg819ldq,ieg819ldq_flc.fits,16398,/grp/hst/wfc3u/postflash_2021/mastDownload/HST...,2021-03-23,16:48:36,A,7310,100.0,MED,False,0.5,4864.834473,4862.593875,4934.711914,4934.71,447.871613,429.116146,48.625939,2021-03-23 16:48:36
ic6de3riq,ic6de3riq_flc.fits,13078,/grp/hst/wfc3u/postflash_2021/mastDownload/HST...,2013-03-09,05:53:22,B,22,0.3,MED,True,0.5,13.109229,12.910655,12.831968,12.832,11.218914,3.541943,43.035518,2013-03-09 05:53:22


In [ ]:
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)] 
fullframe_pf_A = postflash_data.loc[(postflash_data['subarray'] == False) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_cur'] == 'MED') & (postflash_data['flash_dur'] == 100.0)] 
fullframe_pf_B = postflash_data.loc[(postflash_data['subarray'] == False) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_cur'] == 'MED') & (postflash_data['flash_dur'] == 100.0)]
fullframe_pf_A

In [ ]:
# normalized to the mean of first observation
sorted_x = fullframe_pf_A['datetime'].sort_values()
first_rootname = sorted_x.index[0]
x = Time(sorted_x,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(fullframe_pf_A['sig_mean'])
y = fullframe_pf_A['sig_mean']/fullframe_pf_A['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print(popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean of First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(sorted_x, y,size = 4, legend_label = "Med fullframe Shutter A")
p.line(sorted_x,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
sorted_x = fullframe_pf_B['datetime'].sort_values()
first_rootname = sorted_x.index[0]
x = Time(sorted_x,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(fullframe_pf_B['sig_mean'])
y = fullframe_pf_B['sig_mean']/fullframe_pf_B['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print(popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean of First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(sorted_x, y,size = 4, legend_label = "Med fullframe Shutter B")
p.line(sorted_x,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)